# Data Wrangling4
jdjhksjhdhd

In [ ]:
Area of Interest

## CopDem
Data manipulation to download, reproject/resampling (3031, 32 ) and mosaic for further use in the analysis. Note the Dem will bve further reprojected to match each DEM we are comparing againts

In [ ]:
## code to create a continenta;l raster for CopDEM
## Produce 3x tiffs for CopDEM reprojected to 3031 with resolutions 32, 10 and 2m for one area of interest.

import os
import glob
from osgeo import gdal

##DEM Stitcher modules
import yaml
import argparse
import os
import asf_search as asf
from eof.download import download_eofs #Install
import logging
import zipfile
from shapely.geometry import Polygon, box
import rasterio
from dem_stitcher import stitch_dem
import docker
from utils import * #Install
from etad import * #Install
import time
import shutil
import json
import geopandas as gpd


# Define the paths to your input folder and output folder
input_folder = '/g/data/yp75/Projects/DEAnt_DEM_Eval/DEMs/forAlex/CopDEM30_AOI_01'
output_folder = '/g/data/yp75/Projects/DEAnt_DEM_Eval/DEMs/forAlex/CopDEM30_AOI_01/Output'

# Define the target projection (EPSG:3031)
target_srs = 'EPSG:3031'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

#List all raster files in the input folder
raster_files = glob.glob(f'{input_folder}/**/*.tif', recursive=True)

##DEM Stitcher Step to output mosaic



for raster_file in raster_files:
    input_raster_path = os.path.join(input_folder, raster_file)
    output_raster_path = os.path.join(output_folder, raster_file)

    # Open the input raster
    input_raster = gdal.Open(input_raster_path)

    # Reproject the raster
    # gdal.Warp(output_raster_path, input_raster, dstSRS=target_srs, dstNodata=-9999, options=['-t_srs', target_srs, '-to', 'VERT_DATUM=' + vertical_datum])

 # Reproject the raster
    gdal.Warp(output_raster_path, input_raster, dstSRS=target_srs, dstNodata=-9999, options=['-t_srs', target_srs, '-to'])

    # Close the input raster
    input_raster = None

    print(f"Reprojected raster saved to {output_raster_path}")

print("All rasters have been reprojected.")




## REMA 32


In [ ]:
#code for Rema 32
# 
import pystac
import requests
import geopandas as gpd
from shapely.geometry import box
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from rasterio.plot import show

# Define the bounding box for the area of interest (xmin, ymin, xmax, ymax)
bounding_box = (-180, -90, 180, 90)  # Example: World bounding box
min_res = 0.0001  # Define the minimum resolution threshold, e.g., 0.0001 degrees

# URL of the REMA DEM STAC catalog or API endpoint (replace with actual endpoint)
stac_catalog_url = "https://remadem.stac.earth/catalog.json"

# Load the STAC catalog
catalog = pystac.Catalog.from_file(stac_catalog_url)

# Function to search for DEM data based on bounding box and resolution
def search_rema_dem(catalog, bbox, min_res):
    # Create a bounding box geometry
    geom = box(*bbox)
    
    # Initialize a list to store filtered items
    filtered_items = []

    # Iterate over the collections in the catalog
    for collection in catalog.get_all_collections():
        for item in collection.get_items():
            # Filter based on bounding box
            if geom.intersects(item.geometry):
                # Get item metadata (resolution is typically stored in the asset or item properties)
                asset = item.assets.get('elevation')  # Adjust key based on asset type
                if asset:
                    # Fetch the resolution from the asset metadata (example)
                    asset_res = asset.metadata.get('res', None)
                    if asset_res and asset_res <= min_res:
                        filtered_items.append(item)
    
    return filtered_items

# Search for DEM items within the bounding box and resolution threshold
dem_items = search_rema_dem(catalog, bounding_box, min_res)

# Example of how to print out the selected DEM items
for item in dem_items:
    print(f"Item ID: {item.id}")
    print(f"Title: {item.title}")
    print(f"Geometry: {item.geometry}")
    print("-----------")

# If we found DEM items, download and plot one
if dem_items:
    # Get the first item (you can choose a different item or loop over them)
    item = dem_items[0]
    asset = item.assets['elevation']
    
    # Get the URL of the DEM file (replace this with the correct file URL or asset URL)
    dem_url = asset.href
    
    # Download the DEM file (or use a local file path if you already have it)
    with requests.get(dem_url, stream=True) as r:
        with open("rema_dem.tif", "wb") as f:
            f.write(r.content)

    # Open the DEM file with rasterio
    with rasterio.open("rema_dem.tif") as src:
        # Read the data
        dem_data = src.read(1)  # Reading the first band
        
        # Plot the DEM data using matplotlib
        plt.figure(figsize=(10, 6))
        show(src, title="REMA DEM of Antarctica")
        plt.colorbar(label="Elevation (meters)")
        plt.show()
else:
    print("No DEM items found that match the criteria.") 

In [ ]:
#code for CopDEM resampling to Reema32 grid